## Task Group 1 - Import and Inspect

### Task 1

Import the CSV file `internet.csv` and assign it to the variable `internet`. Preview the first few rows.

In [1]:
# Import relevant libraries
import pandas as pd

# Import internet dataset and preview first few rows
internet = pd.read_csv('internet.csv')
internet.head()

,entity,code,year,internet_users_per_100
0,Afghanistan,AFG,1990,0.0
1,Afghanistan,AFG,1991,0.0
2,Afghanistan,AFG,1992,0.0
3,Afghanistan,AFG,1993,0.0
4,Afghanistan,AFG,1994,0.0


<details>
    <summary style="display:list-item; font-size:16px; color:blue;"><i>What is the structure of this dataset? Toggle to check!</i></summary>

Each row of the dataset records the percent of a country's population that uses the internet. According to Our World in Data, a person is defined as an `internet user` by the International Telecommunication Union if they accessed the internet from any location in the last three months.

The columns of the dataset are
- `entity`, the name of the country, region, or income bracket
- `code`, the three-letter country code
- `year`, the year
- `internet_users_per_100`, the number of internet users for every 100 people in the entity's population

Note that we have already trimmed the source dataset to 1990-2019.
</details>

 ### Task 2

The column `internet_users_per_100` is well-named, but let's shorten it.

Clean the dataset by updating the column name `internet_users_per_100` to `percent_online`. Print out `.info()` after doing so. Are there any incorrect data types or missing data issues to be concerned about before diving in?

In [2]:
internet = internet.rename({'internet_users_per_100':'percent_online'},axis=1)
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6367 entries, 0 to 6366
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   entity          6367 non-null   object 
 1   code            6367 non-null   object 
 2   year            6367 non-null   int64  
 3   percent_online  6367 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 199.1+ KB


## Task Group 2 - Years to Reach Mainstream Use

### Task 3

Let's analyze how long it takes for internet use to reach the majority of people in an `entity`.

Write a function that takes a `row` as input and classifies it by `percent_online`:
- `0` should be classified as `none`
- greater than 0 and under 25 should be classified as `few`
- 25-50 should be classified as `some`
- over 50 should be classified as `most`


In [3]:
def internet_reach(row):
    if row == 0:
        return 'none'
    elif row < 25:
        return 'few'
    elif row > 50:
        return 'most'
    else:
        return 'some'

### Task 4

Create a new column named `amount` on `internet` by applying the function from Task 3 to each row.

In [4]:
internet['amount'] = internet['percent_online'].apply(internet_reach)
print(internet.head())

        entity code  year  percent_online amount
0  Afghanistan  AFG  1990             0.0   none
1  Afghanistan  AFG  1991             0.0   none
2  Afghanistan  AFG  1992             0.0   none
3  Afghanistan  AFG  1993             0.0   none
4  Afghanistan  AFG  1994             0.0   none


### Task 5

Let's figure out the first year in our data that each entity reached each level of internet use.

Pivot `internet` using
- `year` as the values
- `entity` and `code` as the index
- `amount` as the column
- `min` as the aggfunc

Reset the index and save the result as `years`.

In [5]:
years = pd.pivot_table(internet,
               index=['entity','code'],
               columns='amount',
               values='year',
               aggfunc='min').reset_index()

years.head()

amount,entity,code,few,most,none,some
0,Afghanistan,AFG,2001.0,NaN,1990.0,NaN
1,Albania,ALB,1995.0,2013.0,1990.0,2009.0
2,Algeria,DZA,1994.0,2019.0,1990.0,2014.0
3,American Samoa,ASM,NaN,NaN,1990.0,NaN
4,Andorra,AND,1996.0,2007.0,1990.0,2004.0


### Task 6

Albania took from 1995-2009 to reach 25%+ of the country online, corresponding to 14 years. 

By contrast, it only took from 2009-2013, 4 years, to reach over 50%. That's over 3 times faster!

Let's see if this phenomenon holds in general.

Create two new columns:
- `few2some` should be the number of years between `few` and `some`
- `some2most` should be the number of years between `some` and `most`

In [6]:
years['few2some'] = years['some'] - years['few']
years['some2most'] = years['most'] - years['some']
years.head()

amount,entity,code,few,most,none,some,few2some,some2most
0,Afghanistan,AFG,2001.0,NaN,1990.0,NaN,NaN,NaN
1,Albania,ALB,1995.0,2013.0,1990.0,2009.0,14.0,4.0
2,Algeria,DZA,1994.0,2019.0,1990.0,2014.0,20.0,5.0
3,American Samoa,ASM,NaN,NaN,1990.0,NaN,NaN,NaN
4,Andorra,AND,1996.0,2007.0,1990.0,2004.0,8.0,3.0


### Task 7

Print out the average of `few2some` and the average of `some2most`.

In [7]:
avg_few2some = years['few2some'].mean()
avg_some2most = years['some2most'].mean()
print(f"Average of few2some {avg_few2some} and average of some2most {avg_some2most}")

Average of few2some 14.065573770491802 and average of some2most 4.943262411347518


<details>
    <summary style="display:list-item; font-size:16px; color:blue;"><i>What did we learn about the spread of internet use? Toggle to check!</i></summary>

Of the countries in the dataset that have passed the 25% threshold, it takes on average `14` years to go from non-zero internet use to over `25%`. But then, for countries that pass 25% it takes only `5` more years on average to reach a majority of the country.
    
There are some ways that this analysis could be improved. If a country "passes" 25% by jumping suddenly to 45%, for example, it wouldn't be surprising that it takes less time to pass 50%. Think about how you could refine our analysis to take this into account!
</details>

### Task 8

This dataset includes some rows that correspond to broader geographic regions and income brackets, not just countries. These are labeled `CAT` in the `code` column.

Display `years` filtered to rows where `code` is `CAT`. Sort by `few2some`, ascending.

In [8]:
filtered_years = years[years['code'] == 'CAT'].sort_values(by='few2some', ascending=True)
print(filtered_years)

amount                        entity code     few    most    none    some  \
151                    North America  CAT  1990.0  2001.0     NaN  1998.0   
87                       High income  CAT  1990.0  2004.0     NaN  2000.0   
66                    European Union  CAT  1990.0  2006.0     NaN  2002.0   
65           Europe and Central Asia  CAT  1990.0  2009.0     NaN  2003.0   
109      Latin America and Caribbean  CAT  1991.0  2015.0  1990.0  2008.0   
56             East Asia and Pacific  CAT  1990.0  2016.0     NaN  2008.0   
218              Upper middle income  CAT  1991.0  2015.0  1990.0  2009.0   
133     Middle East and North Africa  CAT  1990.0  2016.0     NaN  2010.0   
117            Low and middle income  CAT  1991.0     NaN  1990.0  2012.0   
134                    Middle income  CAT  1991.0     NaN  1990.0  2012.0   
119              Lower middle income  CAT  1992.0     NaN  1990.0  2017.0   
189                       South Asia  CAT  1992.0     NaN  1990.0  2019.0   

<details>
    <summary style="display:list-item; font-size:16px; color:blue;"><i>What did we learn about the spread of internet use? Toggle to check!</i></summary>

Perhaps unsurprisingly, we see a pretty strong disparity in internet use. The global north and higher income brackets tend to have reached the `most` category fairly quickly. At least in this data, low/middle income brackets and southern regions like South Asia have yet to reach the `most` category, and take the longest to go from `few` to `some`.
</details>

## Task Group 3 - Growth by Decade

### Task 9

Let's return to our original `internet` dataset and investigate how quickly internet use has increased over the decades. 

Note that we don't have full data for all countries. For example, for Nauru we only have one year of data in the 2000s and two years of data in the 2010s. For this project, we'll use the rough approximation of analyzing growth over the years we do have, but to make any strong scientific conclusions we'd want to refine this analysis.

Most of the missing years are in the 1990s, so start by filtering the DataFrame down to only years past 1999.

In [18]:
internet = internet[internet['year'] > 1999]
internet.head()

,entity,code,year,percent_online,amount
6,Afghanistan,AFG,2001,0.004723,few
7,Afghanistan,AFG,2002,0.004561,few
8,Afghanistan,AFG,2003,0.087891,few
9,Afghanistan,AFG,2004,0.105809,few
10,Afghanistan,AFG,2005,1.224148,few


### Task 10

Now let's add decade information!

Create a function named `decader` that takes a `row` as input and outputs the decade corresponding to `row['year']`.

The format for the decade should be as follows:

- the years `2000-2009` inclusive should have decade `2000s`
- the years `2010-2019` inclusive should have decade `2010s`

In [21]:
def decader(row):
    decade = str(row['year'])
    decade= decade[0:3]+'0s'
    return decade

# Test the function by checking for first row
decader(internet.iloc[0,:])

'2000s'

### Task 11

Use `.apply()` to create a new column `decade` that stores decade information for each row.

In [22]:
internet['decade'] = internet.apply(decader,axis=1)
internet.head()

,entity,code,year,percent_online,amount,decade
6,Afghanistan,AFG,2001,0.004723,few,2000s
7,Afghanistan,AFG,2002,0.004561,few,2000s
8,Afghanistan,AFG,2003,0.087891,few,2000s
9,Afghanistan,AFG,2004,0.105809,few,2000s
10,Afghanistan,AFG,2005,1.224148,few,2000s


### Task 12

Now, let's write a function to calculate how much internet use has changed over a sequence of years.

Write a function `change` that takes a DataFrame `column` as input.

If the `len(column)` is `1` (that is, the column has only one entry), return that entry (`.iloc[0]`)

Otherwise, return the difference between the last entry (`.iloc[-1]`) and the first entry (`.iloc[0]`).

In [23]:
def change(column):
    if len(column) == 1:
        return column.iloc[0]
    else:
        return(column.iloc[-1] - column.iloc[0])

### Task 13

In order for `change` to compute the change from the start of a decade to its end, we need the years to be in order, so that the last entry of the column is the latest year of the decade in the dataset.

Sort `internet` by `year`, ascending.

In [25]:
internet = internet.sort_values(by='year', ascending=True)
internet.head()

,entity,code,year,percent_online,amount,decade
4993,Saudi Arabia,SAU,2000,2.210692,few,2000s
5454,Sudan,SDN,2000,0.025785,few,2000s
2905,Kazakhstan,KAZ,2000,0.668594,few,2000s
2462,High income,CAT,2000,30.545069,some,2000s
5424,Sub-Saharan Africa,CAT,2000,0.500951,few,2000s


### Task 14

Let's apply `change` to calculate how much the internet has grown over each decade. At the same time, let's calculate the first and last year of data for each country, so we can know how many years it took for that change to occur.

Group `internet` by `entity` and `decade`. Using `.agg`:

- apply `change` to `percent_online`
- apply `['min','max']` to `year`

Assign the result to the variable `decade_growth`, and reset the index.

In [31]:
decade_growth = internet.groupby(['entity','decade']).agg(
    {'percent_online':change,
     'year': ['min','max']}).reset_index()
decade_growth.head()

entity decade percent_online  year      
                              change   min   max
0  Afghanistan  2000s       3.545277  2001  2009
1  Afghanistan  2010s      13.600000  2010  2019
2      Albania  2000s      41.085903  2000  2009
3      Albania  2010s      23.550392  2010  2019
4      Algeria  2000s      10.738294  2000  2009

### Task 15

Let's flatten the columns of this new DataFrame.

Reassign `.columns` using the names `['entity','decade','change','min','max']`

In [32]:
decade_growth.columns = ['entity','decade','change','min','max']
decade_growth.head()

,entity,decade,change,min,max
0,Afghanistan,2000s,3.545277,2001,2009
1,Afghanistan,2010s,13.600000,2010,2019
2,Albania,2000s,41.085903,2000,2009
3,Albania,2010s,23.550392,2010,2019
4,Algeria,2000s,10.738294,2000,2009


### Task 16

Let's calculate yearly growth.

Add a new column `annual` by calculating `decade_growth['change']` divided by `decade_growth['max'] - decade_growth['min']`

In [34]:
decade_growth['annual'] = decade_growth['change'] / (decade_growth['max'] - decade_growth['min'])
decade_growth.head()

,entity,decade,change,min,max,annual
0,Afghanistan,2000s,3.545277,2001,2009,0.443160
1,Afghanistan,2010s,13.600000,2010,2019,1.511111
2,Albania,2000s,41.085903,2000,2009,4.565100
3,Albania,2010s,23.550392,2010,2019,2.616710
4,Algeria,2000s,10.738294,2000,2009,1.193144


### Task 17

Let's make this table a bit more human-readable by converting to wide format.

Pivot `decade_growth` using
- `annual` as the values
- `entity` as the index
- `decade` as the columns

Assign the result back to `decade_growth`.

In [35]:
decade_growth = pd.pivot_table(decade_growth,
              values='annual',
              columns='decade',
              index='entity'
              )
decade_growth.head()


decade,2000s,2010s
entity,,
Afghanistan,0.443160,1.511111
Albania,4.565100,2.616710
Algeria,1.193144,5.044445
Andorra,7.554574,1.509638
Angola,0.243884,3.288889


### Task 18

Create a new column `ratio` that takes `2010s` and divides by `2000s`.

In [36]:
decade_growth['ratio'] = decade_growth['2010s'] / decade_growth['2000s']
decade_growth.head()

decade,2000s,2010s,ratio
entity,,,
Afghanistan,0.443160,1.511111,3.409857
Albania,4.565100,2.616710,0.573199
Algeria,1.193144,5.044445,4.227860
Andorra,7.554574,1.509638,0.199831
Angola,0.243884,3.288889,13.485475


### Task 19

Print out descriptive statistics of the new `ratio` column. What do you notice?

In [37]:
decade_growth['ratio'].describe()

count    220.000000
mean       5.119733
std       14.289239
min        0.000000
25%        0.718154
50%        1.579371
75%        4.226106
max      140.184335
Name: ratio, dtype: float64

<details>
    <summary style="display:list-item; font-size:16px; color:blue;"><i>What did we learn about internet use? Toggle to check!</i></summary>

On average, annual growth in the 2010s was 5 times as large as annual growth in the 2000s.
    
This average is being pulled up by some enormous outliers. The median is `1.5`, so half the entities in the dataset had over 1.5x more annual growth in the 2010s than the 2000s.
    
Interestingly, the 25th percentile is `.71`, so over 25% of countries experienced less annual growth in the 2010s than in the 2000s! This likely makes sense for countries like the USA, which had already reached a majority of the population by 2010. What other factors might cause a country to end up with slower internet growth in the 2010s than in the 2000s?
    
Again, for some countries we may be missing some decade information. We encourage you dig deeper into this dataset if you want to refine our analysis!
   
</details>

There is so much more to discover in this dataset. Feel free to add more cells below to extend the analysis, and happy coding!